## Создаём словарь для CondBERT'а

In [8]:
!git clone https://github.com/s-nlp/detox
! pip install -r detox/requirements.txt -q

fatal: destination path 'detox' already exists and is not an empty directory.


In [9]:
import os
import sys

def add_sys_path(p):
    p = os.path.abspath(p)
    print(p)              
    if p not in sys.path:
        sys.path.append(p)

In [10]:
# adding the path to the condebert code root
add_sys_path('detox/emnlp2021/style_transfer/condBERT')

/kaggle/working/detox/emnlp2021/style_transfer/condBERT


In [11]:
from importlib import reload

In [12]:
import condbert
reload(condbert)
from condbert import CondBertRewriter

In [13]:
!pip install flair keras_preprocessing

In [14]:
import choosers
reload(choosers)
from choosers import EmbeddingSimilarityChooser

In [15]:
import multiword
reload(multiword)
from multiword.masked_token_predictor_bert import MaskedTokenPredictorBert

# 0. Prerequisites

In [23]:
path_to_data = '/kaggle/input/bias-data/bias_data/WNC/biased.full'

In [18]:
VOCAB_DIRNAME = 'bias_data/WNC'

# 1. Loading BERT

In [19]:
!pip install transformers

In [20]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import numpy as np
import pickle
import os
from tqdm.auto import tqdm, trange

In [21]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['WANDB_DISABLED'] = 'True'
device = torch.device('cuda:0')
device = torch.device('cpu')

In [22]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name, model_max_length=128)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [24]:
model = BertForMaskedLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
model.to(device);

# 2. Preparing the vocabularires.


- negative-words.txt
- positive-words.txt
- word2coef.pkl
- token_biasedness.txt

These files should be prepared once. 

In [26]:
biased_corpus_path = '/kaggle/input/bias-data/bias_data/WNC/biased.full'

In [27]:
import pandas as pd

In [28]:
pd.set_option('display.max_colwidth', -1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [29]:
colnames = ['id','src_tok','tgt_tok','src_raw',
            'tgt_raw','src_POS_tags','tgt_parse_tags']
biased_df = pd.read_csv(biased_corpus_path, 
                        sep='\t', on_bad_lines='skip', names=colnames)
biased_df[['src_raw','tgt_raw']]

,src_raw,tgt_raw
0,"during the campaign, controversy erupted over alleged differences between palin's positions as a gubernatorial candidate and her position as a vice-presidential candidate.","during the campaign, some pointed out alleged differences between palin's positions as a gubernatorial candidate and her position as a vice-presidential candidate."
1,nicaea was convoked by the emperor constantine in mayaugust 325 to address the arian heresy that christ is of a distinct substance from the father.,nicaea was convoked by the emperor constantine in mayaugust 325 to address the arian position that christ is of a distinct substance from the father.
2,"it was rather unfortunate that he vehemently opposed the budding indian scientist subrahmanyan chandrasekhar about his theory on the maximum mass of stars known as white dwarfs, the mass above which the star collapses and becomes a neutron star, quark star or black hole.","he vehemently opposed the budding indian scientist subrahmanyan chandrasekhar about his theory on the maximum mass of stars known as white dwarfs, the mass above which the star collapses and becomes a neutron star, quark star or black hole."
3,"dennis the menace is an american animated series produced by dic entertainment, based on the classic comic strip by hank ketcham.","dennis the menace is an american animated series produced by dic entertainment, based on the comic strip by hank ketcham."
4,"today, on large farms, motorcycles, dogs or men on horseback may herd sheep.","today, on large farms, motorcycles, dogs or people on horseback may herd sheep."
...,...,...
181469,"the most famous of these is dale earnhardt, who died february 18, 2001.","arguably the most notable was dale earnhardt, who died february 18, 2001."
181470,"because of the purity of the ancient breed, the akhal-teke has been used for developing new breeds, most recently the nez perce horse (appaloosa x akhal-teke).","because of the genetic prepotency of the ancient breed, the akhal-teke has been used for developing new breeds, most recently the nez perce horse (appaloosa x akhal-teke)."
181471,"influenced by serge gainsbourg, the velvet underground, jean luc godard, gang of four, john fante, joy division, jacques brel and johnny cash , john and jehn's music is a collection of sonic experiments, carried by dark humour on razor-sharp pop melodies.","influenced by serge gainsbourg, the velvet underground, jean luc godard, gang of four, john fante, joy division, jacques brel and johnny cash , john and jehn's music is a collection of sonic experiments, carried by dark humour on pop melodies."
181472,"northern indiana, for example, contains some of the most beautiful beaches found in any of the five great lakes , and many of the chicago area's trails and roads follow some of these ancient beach lines or ridges in the sand spits.","northern indiana, for example, contains the indiana dunes , and many of the chicago area's trails and roads follow some of these ancient beach lines or ridges in the sand spits."


In [30]:
with open('biased_df.txt', 'w') as f_out:
    f_out.write('\n'.join(biased_df['src_raw']))

In [31]:
with open('norm_df.txt', 'w') as f_out:
    f_out.write('\n'.join(biased_df['tgt_raw']))

In [32]:
if not os.path.exists(VOCAB_DIRNAME):
    os.makedirs(VOCAB_DIRNAME)

In [33]:
biased_corpus_path = 'biased_df.txt'
norm_corpus_path = 'norm_df.txt'

### 2.1 Preparing the DRG-like vocabularies

In [34]:
import os
import argparse
import numpy as np
from tqdm import tqdm
from nltk import ngrams
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer



class NgramSalienceCalculator():
    def __init__(self, biased_corpus, norm_corpus, use_ngrams=False):
        ngrams = (1, 3) if use_ngrams else (1, 1)
        self.vectorizer = CountVectorizer(ngram_range=ngrams)

        biased_count_matrix = self.vectorizer.fit_transform(biased_corpus)
        self.biased_vocab = self.vectorizer.vocabulary_
        self.biased_counts = np.sum(biased_count_matrix, axis=0)

        norm_count_matrix = self.vectorizer.fit_transform(norm_corpus)
        self.norm_vocab = self.vectorizer.vocabulary_
        self.norm_counts = np.sum(norm_count_matrix, axis=0)

    def salience(self, feature, attribute='biased', lmbda=0.5):
        assert attribute in ['biased', 'norm']
        if feature not in self.biased_vocab:
            biased_count = 0.0
        else:
            biased_count = self.biased_counts[0, self.biased_vocab[feature]]

        if feature not in self.norm_vocab:
            norm_count = 0.0
        else:
            norm_count = self.norm_counts[0, self.norm_vocab[feature]]

        if attribute == 'biased':
            return (biased_count + lmbda) / (norm_count + lmbda)
        else:
            return (norm_count + lmbda) / (biased_count + lmbda)


In [35]:
from collections import Counter
c = Counter()

for fn in tqdm([biased_corpus_path, norm_corpus_path]):
    with open(fn, 'r') as corpus:
        for line in corpus.readlines():
            for tok in line.strip().split():
                c[tok] += 1

print(len(c))

100%|██████████| 2/2 [00:05<00:00,  2.56s/it]

320412


In [36]:
vocab = {w for w, _ in c.most_common() if _ > 0}  # if we took words with > 1 occurences, vocabulary would be x2 smaller, but we'll survive this size
print(len(vocab))

320412


In [37]:
with open(biased_corpus_path, 'r') as biased_corpus, open(norm_corpus_path, 'r') as norm_corpus:
    corpus_biased = [' '.join([w if w in vocab else '<unk>' for w in line.strip().split()]) for line in biased_corpus.readlines()]
    corpus_norm = [' '.join([w if w in vocab else '<unk>' for w in line.strip().split()]) for line in norm_corpus.readlines()]

In [38]:
neg_out_name = 'biased-words.txt' 
pos_out_name = 'neut-words.txt'

In [39]:
threshold = 4

In [40]:
sc = NgramSalienceCalculator(corpus_biased, corpus_norm, False)
seen_grams = set()

with open(neg_out_name, 'w') as neg_out, open(pos_out_name, 'w') as pos_out:
    for gram in tqdm(set(sc.biased_vocab.keys()).union(set(sc.norm_vocab.keys()))):
        if gram not in seen_grams:
            seen_grams.add(gram)
            biased_salience = sc.salience(gram, attribute='biased')
            neut_salience = sc.salience(gram, attribute='norm')
            if biased_salience > threshold:
                neg_out.writelines(f'{gram}\n')
            elif neut_salience > threshold:
                pos_out.writelines(f'{gram}\n')

100%|██████████| 154351/154351 [00:02<00:00, 70027.39it/s]


## 2.2 Evaluating word biasedness with a logistic regression

In [41]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=1000))

In [42]:
X_train = corpus_biased + corpus_norm
y_train = [1] * len(corpus_biased) + [0] * len(corpus_norm)
pipe.fit(X_train, y_train);

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [43]:
coefs = pipe[1].coef_[0]
coefs.shape

(154351,)

In [44]:
word2coef = {w: coefs[idx] for w, idx in pipe[0].vocabulary_.items()}

In [49]:
import pickle
with open('word2coef.pkl', 'wb') as f:
    pickle.dump(word2coef, f)

## 2.3 Labelling BERT tokens by biasedness

In [47]:
from collections import defaultdict
bias_counter = defaultdict(lambda: 1)
nonbias_counter = defaultdict(lambda: 1)

for text in tqdm(corpus_biased):
    for token in tokenizer.encode(text[:256]):
        bias_counter[token] += 1
for text in tqdm(corpus_norm):
    for token in tokenizer.encode(text):
        nonbias_counter[token] += 1

100%|██████████| 181474/181474 [03:00<00:00, 1005.84it/s]


In [50]:
token_biasedness = [bias_counter[i] / (nonbias_counter[i] + bias_counter[i]) for i in range(len(tokenizer.vocab))]

In [51]:
with open('token_biasedness.txt', 'w') as f:
    for t in token_biasedness:
        f.write(str(t))
        f.write('\n')

# 3. Setting up the model

### 3.1 Loading the vocabularies

In [54]:
with open("biased-words.txt", "r") as f:
    s = f.readlines()
negative_words = list(map(lambda x: x[:-1], s))

with open("neut-words.txt", "r") as f:
    s = f.readlines()
positive_words = list(map(lambda x: x[:-1], s))

In [55]:
import pickle
with open('word2coef.pkl', 'rb') as f:
    word2coef = pickle.load(f)

In [56]:
token_biasedness = []
with open('token_biasedness.txt', 'r') as f:
    for line in f.readlines():
        token_biasedness.append(float(line))
token_biasedness = np.array(token_biasedness)
token_biasedness = np.maximum(0, np.log(1/(1/token_biasedness-1)))   # log odds ratio

# discourage meaningless tokens
for tok in ['.', ',', '-']:
    token_biasedness[tokenizer.encode(tok)][1] = 3

for tok in ['you']:
    token_biasedness[tokenizer.encode(tok)][1] = 0

In [58]:
def adjust_logits(logits, label=0):
    return logits - token_biasedness * 100 * (1 - 2 * label)

predictor = MaskedTokenPredictorBert(model, tokenizer, max_len=250, 
                                     device=device, label=0, contrast_penalty=0.0, 
                                     logits_postprocessor=adjust_logits, confuse_bert_args=True)

editor = CondBertRewriter(
    model=model,
    tokenizer=tokenizer,
    device=device,
    neg_words=negative_words,
    pos_words=positive_words,
    word2coef=word2coef,
    token_toxicities=token_biasedness,
    predictor=predictor,
)

The model below is used for reranking BERT hypotheses and helps to increase semantic similarity by choosing the hypotheses with  embeddings similar to the orignal words. 

In [59]:
chooser = EmbeddingSimilarityChooser(sim_coef=10, tokenizer=tokenizer)

2023-03-29 08:19:36,542 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpv1inmia2


100%|██████████| 160000128/160000128 [00:08<00:00, 19681692.16B/s]

2023-03-29 08:19:45,084 copying /tmp/tmpv1inmia2 to cache at /root/.flair/embeddings/glove.gensim.vectors.npy
2023-03-29 08:19:45,258 removing temp file /tmp/tmpv1inmia2


2023-03-29 08:19:45,682 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmp_rc_57gp


100%|██████████| 21494764/21494764 [00:01<00:00, 12083291.23B/s]

2023-03-29 08:19:47,867 copying /tmp/tmp_rc_57gp to cache at /root/.flair/embeddings/glove.gensim
2023-03-29 08:19:47,902 removing temp file /tmp/tmp_rc_57gp


# 4. Finally, the inference

Parallel application of the model to all tokens, fast, but dirty. 

Application of the model to all the tokens sequentially, in the multiword mode. 

Parameters that could be tuned:
* The coeffincient in `adjust_logits` - the larger it is, the more the model avoids biased words
* The coefficient in `EmbeddingSimilarityChooser` - the larger it is, the more the model tries to preserve content 
* n_tokens - how many words can be generated from one
* n_top - how many BERT hypotheses are reranked

### Возьмём переведём 1000 примеров, чтобы оценить качество

In [61]:
test_path = '/kaggle/input/bias-data/bias_data/WNC/biased.word.test'

In [65]:
colnames = ['id','src_tok','tgt_tok','src_raw',
            'tgt_raw','src_POS_tags','tgt_parse_tags']
test_df = pd.read_csv(test_path, 
                        sep='\t', on_bad_lines='skip', names=colnames)
test_df[['src_raw','tgt_raw']]

,src_raw,tgt_raw
0,"in april 2009 a brazilian human rights group, torture never again, awarded the five its chico mendes medal, because their rights had been violated.","in april 2009 a brazilian human rights group, torture never again, awarded the five its chico mendes medal, under the pretext that their rights had been violated."
1,"the 51 day standoff and ensuing murder of 76 men, women, and children--the branch davidians--in waco, texas.","the 51 day standoff and ensuing deaths of 76 men, women, and children--the branch davidians--in waco, texas."
2,"mark oaten (born 8 march 1964, watford) is a disgraced liberal democrat politician in the united kingdom, and member of parliament for the winchester constituency.","mark oaten (born 8 march 1964, watford) is a liberal democrat politician in the united kingdom, and member of parliament for the winchester constituency."
3,another infamous period of colonisation in ancient times was from the romans.,another period of colonisation in ancient times was from the romans.
4,photo sequence of astonishing 2005 chicagoland crash with ryan briscoe.,photo sequence of 2005 chicagoland crash with ryan briscoe.
...,...,...
995,especially as the cost to refurbish and modify a used vintage lenco turntable is small in comparison with the cost of buying a new turntable (provided that one has invested in the tools needed to tackle such a projectchief among these a table-saw and router).,especially as the cost to refurbish and modify a used vintage lenco turntable can be small in comparison with the cost of buying a new turntable (provided that one has invested in the tools needed to tackle such a projectchief among these a table-saw and router).
996,blood libels against jews are incorrect accusations that jews use human blood in certain aspects of their religious rituals and holidays.,blood libels against jews are accusations that jews use human blood in certain aspects of their religious rituals and holidays.
997,"jacob collier (born august 2, 1994) is a singer, arranger, composer, and prodigious multi-instrumentalist based in london, england.","jacob collier (born august 2, 1994) is a singer, arranger, composer, and multi-instrumentalist based in london, england."
998,they immediately decided to apply it to the annoying netnews group.,they immediately decided to apply it to the net.singles netnews group.


## Переводим 1000 предложений с confuse_bert_args=True

In [66]:
def adjust_logits(logits, label=0):
    return logits - token_biasedness * 100 * (1 - 2 * label)

predictor = MaskedTokenPredictorBert(model, tokenizer, max_len=128, 
                                     device=device, label=0, contrast_penalty=0.0, 
                                     logits_postprocessor=adjust_logits, confuse_bert_args=True)

editor = CondBertRewriter(
    model=model,
    tokenizer=tokenizer,
    device=device,
    neg_words=negative_words,
    pos_words=positive_words,
    word2coef=word2coef,
    token_toxicities=token_biasedness,
    predictor=predictor,
)

In [67]:
def translate(sent):
    return editor.replacement_loop(sent.strip(), verbose=False, 
                                 chooser=chooser, n_tokens=(1, 2, 3), n_top=10)

In [103]:
with open('outputs_bias1.txt', 'wb') as res:
    for sent in tqdm(test_df['src_raw']):
        res.writelines(f'{translate(sent)}\n')

## Переводим 1000 предложений с confuse_bert_args=False

In [81]:
def adjust_logits(logits, label=0):
    return logits - token_biasedness * 100 * (1 - 2 * label)

predictor = MaskedTokenPredictorBert(model, tokenizer, max_len=128, 
                                     device=device, label=0, contrast_penalty=0.0, 
                                     logits_postprocessor=adjust_logits, confuse_bert_args=False)

editor = CondBertRewriter(
    model=model,
    tokenizer=tokenizer,
    device=device,
    neg_words=negative_words,
    pos_words=positive_words,
    word2coef=word2coef,
    token_toxicities=token_biasedness,
    predictor=predictor,
)

In [82]:
def translate(sent):
    return editor.replacement_loop(sent.strip(), verbose=False, 
                                 chooser=chooser, n_tokens=(1, 2, 3), n_top=10)

In [104]:
with open('outputs_bias2.txt', 'w') as res:
    for sent in tqdm(book1[205:1000]):
        res.writelines(f'{translate(sent)}\n')

### Примеры модификации 

In [38]:
sentences = ['John McCain exposed as an unprincipled politician',
             'Marriage is a holy union of individuals.',
             'This includes amazing Americans like',
             'The authors expose on nutrition studies.',
             'Jewish forces overcome Arab militants.',
             ]

In [39]:
%%time
for sent in sentences:
  sent = sent.strip()
  first = editor.translate(sent, prnt=False)
  second = translate(sent)
  editor.replacement_loop(sent, verbose=False, 
                                   chooser=chooser, n_tokens=(1, 2, 3), n_top=10)
  print(first, second, '\n', sep ='\n')

john mccain exposed as an unprincipled .

john mccain exposed as an unprincipled democrat .





marriage is a holy union of individuals .

marriage is a holy union of individuals .





this includes all americans like

this includes african - americans like





the authors feature on nutrition studies .

the authors feature on nutrition studies .





. forces overcome arab militants .

israeli defense forces overcome arab rebel forces .





CPU times: user 1min 11s, sys: 134 ms, total: 1min 11s

Wall time: 1min 16s
